In [ ]:
---
creation_date: 2023-10-16
---

| verb | method | notes | 
|---|---|---|
| sort multiindex | `DataFrame.sort_index` | NA |  
| reorder levels | `DataFrame.reorder_levels` | NA | 
| (re)name an axis | `DataFrame.rename_axis` | appears to only work on non-multiindexed axes. No level option |
| (re)name multiindex level(s) | `pd.Index.set_names` | NA |



In [ ]:
import pandas as pd
import numpy as np

arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]


tuples = list(zip(*arrays))

index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])

df = pd.DataFrame(np.random.randn(3, 8), index=["A", "B", "C"], columns=index)
df

## Rename a Multiindex Column Level

The below example renames a multiindexed column level at level 0 through method chaining. The same result can be achieved for renaming a multiindexed index if the axis parameter is changed. The level is specifed by the level parameter, but can also be specified as a string corresponding to an existing name, if there is one.

In [ ]:
# rename a multiindex level

df = (df
      .pipe(lambda df: df.set_axis(df.columns.rename(level=0, names = 'signal'),axis=1))
      )

df

## Reorder Levels

Generally after pivoting and unstacking, A multiindexed column will not be in the desired order. In this case, i wish for 'second' to be the top level category, with 'signal' as the secondary. To achieve this, use '.reorder_levels': 

In [ ]:
df = (
    df
    .reorder_levels(axis=1, order=['second','signal'])  
      )

df

## Sort Multiindexed DataFrame

After transformations, the dataframe tends to be unsorted. For visualization and futher computation, it is best to sort the columns and rows by the top level category. In this example, we want to sort 'second' so all of the columns labelled 'one' are together, and so on:

In [ ]:

df = (
  df
  .sort_index(axis=1)
)
df

## Multiindex Indexing

https://pandas.pydata.org/docs/user_guide/advanced.html

### Partial Indexing

Using a partial label to select a subgroup, dropping the levels of the heirarchical index along the way:

In [ ]:
df['bar']

Only works for the top level inwards:

In [ ]:
df['bar','one']

In [ ]:
try:
    df['one']
except Exception as e:
    display(e)

### Cross Section Indexing

`.xs()` allows row indexing at a specific level, dropping the level indexed on:

In [ ]:
df.xs('one', level='second',axis=1, drop_level=False)

### Indexing by Slice Object

Another method is using `slice` or `pd.IndexSlice`. For example, to subset a multiindex column dataframe to two values of a particular level:

In [ ]:
df

In [ ]:
df.loc[:, pd.IndexSlice[['bar','qux'],:]]

Which excludes 'foo' from the first level while retaining all levels.